# 2. fully-connected

In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

/home/student/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import os

In [3]:
root = '/home/student/project/project-01/nlp_bots/hw-xujing'
pickle_file = 'notMNIST.pickle'
pickle_file = os.path.join(root, pickle_file)
with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save # help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('training set', train_dataset.shape, train_labels.shape)
print('validation set', valid_dataset.shape, valid_labels.shape)
print('test set', test_dataset.shape, test_labels.shape)

training set (200000, 784) (200000, 10)
validation set (10000, 784) (10000, 10)
test set (10000, 784) (10000, 10)


In [5]:
labels = np.arange(6)
print(labels[:, None])
print(labels)

[[0]
 [1]
 [2]
 [3]
 [4]
 [5]]
[0 1 2 3 4 5]


梯度下降 训练

In [6]:
train_subset = 10000

graph = tf.Graph()
with graph.as_default():
    # input data
    # load the training, validtion and test data  into constants that are 
    #attached to the graph
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # variables.
    #there are the parameters that we are going to be training. the weight
    #matrix will be initialized using random values following a (truncated)
    #normal distribution. the biases get initialized to zero.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # training computation.
    #multiply the inputs with the weights matrix, and add biases. we compute 
    #the softmax and cross-entropy (it's one operation in tensorflow, because
    #it's very common, and it can be optimied). we take the average of this
    #cross entropy across all training examples: thatls our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss) # minimum loss with GD
    
    #Predictions for the training, validation, and test data.
    #there are not part of traning, but merely here so that we can report
    #accuracy figuresas we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [7]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 17.720919
Training accuracy: 10.0%
Validation accuracy: 12.1%
Loss at step 100: 2.316870
Training accuracy: 72.0%
Validation accuracy: 69.9%
Loss at step 200: 1.867114
Training accuracy: 74.7%
Validation accuracy: 72.2%
Loss at step 300: 1.627737
Training accuracy: 76.0%
Validation accuracy: 72.7%
Loss at step 400: 1.466738
Training accuracy: 76.9%
Validation accuracy: 73.3%
Loss at step 500: 1.346977
Training accuracy: 77.7%
Validation accuracy: 73.3%
Loss at step 600: 1.252543
Training accuracy: 78.5%
Validation accuracy: 73.6%
Loss at step 700: 1.174945
Training accuracy: 78.9%
Validation accuracy: 73.9%
Loss at step 800: 1.109347
Training accuracy: 79.2%
Validation accuracy: 74.1%
Test accuracy: 82.8%


test accuracy incradibley low, pickle file has bug, need to check

Let's now switch to stochastic gradient descent training instead, which is much faster.

In [8]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    # input data for the training data, we use a placeholder that will be fed
    # at the run time in a batch
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #variables
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    #training computation
    logits =tf.matmul(tf_train_dataset, weights) +biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #prediction for the training, validation, and test data
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) +biases)
    


In [9]:
num_steps = 3001
##3001*128 总共的训练数据   ## 之前是801*10000
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        ##分母减去batch_size是防止当batch_size不能被训练数据量整除时，offset:(offset + batch_size)超出数组界限
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 17.091871
Minibatch accuracy: 6.2%
Validation accuracy: 15.7%
Minibatch loss at step 500: 1.870365
Minibatch accuracy: 72.7%
Validation accuracy: 75.0%
Minibatch loss at step 1000: 1.414816
Minibatch accuracy: 77.3%
Validation accuracy: 76.5%
Minibatch loss at step 1500: 1.114393
Minibatch accuracy: 75.0%
Validation accuracy: 76.4%
Minibatch loss at step 2000: 0.878250
Minibatch accuracy: 77.3%
Validation accuracy: 77.9%
Minibatch loss at step 2500: 0.680358
Minibatch accuracy: 83.6%
Validation accuracy: 77.8%
Minibatch loss at step 3000: 1.439513
Minibatch accuracy: 71.1%
Validation accuracy: 78.9%
Test accuracy: 86.5%


# Problem

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units nn.relu() and 1024 hidden nodes. This model should improve your validation / test accuracy.

In [10]:
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    
    #input data, for the training data we use a placeholder that will be fed when runningwith training minibatch
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #variables
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
    biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
    
    weights2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    #training computation. #shezhi tf.nn.relu()de canshu
    
    lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    
    logits = tf.matmul(lay1_train, weights2) + biases2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    lay1_valid = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    valid_prediction = tf.nn.softmax(tf.matmul(lay1_valid, weights2) +biases2)
    
    lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    test_prediction = tf.nn.softmax(tf.matmul(lay1_test, weights2) + biases2)
  

In [11]:
num_steps = 3001
# 3001*128 xunlishuju zongliang 
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        #pick an offset within the training data, which has been randomized.
        #note: we could use better randomzization across epochs.
        offser = (step * batch_size) % (train_labels.shape[0] - batch_size)
        #generate a minibatch.
        batch_data = train_dataset[offset:(offset +batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        #prepare a dictionary telling the session where to feed the minibath.
        #the key of the dictionary is the placeholder node of the graph to be fed,
        #and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 ==0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 368.156311
Minibatch accuracy: 5.5%
Validation accuracy: 9.5%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 10.0%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 10.0%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 10.0%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 10.0%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 10.0%
Test accuracy: 71.3%


In [12]:
num_steps = 3001
##3001*128 总共的训练数据   ## 之前是801*10000
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 377.823883
Minibatch accuracy: 6.2%
Validation accuracy: 10.5%
Minibatch loss at step 500: 20.845060
Minibatch accuracy: 80.5%
Validation accuracy: 9.7%
Minibatch loss at step 1000: 15.245884
Minibatch accuracy: 82.0%
Validation accuracy: 9.7%
Minibatch loss at step 1500: 5.828614
Minibatch accuracy: 78.1%
Validation accuracy: 9.6%
Minibatch loss at step 2000: 4.185171
Minibatch accuracy: 83.6%
Validation accuracy: 9.8%
Minibatch loss at step 2500: 2.041711
Minibatch accuracy: 87.5%
Validation accuracy: 9.8%
Minibatch loss at step 3000: 6.976463
Minibatch accuracy: 78.9%
Validation accuracy: 9.5%
Test accuracy: 86.7%
